In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder, StandardScaler 

In [2]:
dados = pd.read_table("../datasets/Potencial_Novos_Clientes.txt",
                      sep="\t",
                      header=0)
dados.head()

,COD_CARTAO,IDADE_CLIENTE,RENDA_MENSAL_CLIENTE,BEHAVIOUR_SCORE_CLIENTE,QTD_TRANSACOES_3M,QTD_ITENS_3M,VALOR_GASTO_3M,TICKET_MEDIO_3M,FLAG_ELETRONICOS_3M,SATISFACAO_ULTIMA_COMPRA,VALOR_GASTO_PROX_12M
0,212394,22,1500,52,2,4,802,401.00,0,01_Muito_Satisfeito,226
1,279177,29,1800,94,5,5,1384,276.80,1,03_Neutro,2786
2,291430,36,1500,89,5,6,1610,322.00,1,Nao_Respondeu,3737
3,176612,43,1500,48,3,4,836,278.67,0,01_Muito_Satisfeito,1162
4,223092,28,12090,89,4,5,1318,329.50,0,Nao_Respondeu,2699


#### Transformer

In [3]:
# Podemos criar nossos próprios estimadores e transformers
class ColumnSelector(BaseEstimator, TransformerMixin):
    '''Seleciona um subset de um dado dataframe a partir de uma lista de colunas'''
    def __init__(self, cols_list):
        self.cols_list = cols_list
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.loc[:, self.cols_list]

In [4]:
cols_list_quanti = ['IDADE_CLIENTE',
                  'RENDA_MENSAL_CLIENTE',
                  'BEHAVIOUR_SCORE_CLIENTE',
                  'QTD_TRANSACOES_3M',
                  'QTD_ITENS_3M',
                  'VALOR_GASTO_3M',
                  'TICKET_MEDIO_3M']

dados_subset = ColumnSelector(cols_list_quanti)
dados_subset.transform(dados).head()


,IDADE_CLIENTE,RENDA_MENSAL_CLIENTE,BEHAVIOUR_SCORE_CLIENTE,QTD_TRANSACOES_3M,QTD_ITENS_3M,VALOR_GASTO_3M,TICKET_MEDIO_3M
0,22,1500,52,2,4,802,401.00
1,29,1800,94,5,5,1384,276.80
2,36,1500,89,5,6,1610,322.00
3,43,1500,48,3,4,836,278.67
4,28,12090,89,4,5,1318,329.50


#### Pipeline

In [5]:
num_pipe = Pipeline([('get_num_cols', ColumnSelector(cols_list_quanti)),
                ('fix_nan', SimpleImputer(missing_values=np.nan, strategy='median')),
                ('scale_data', MinMaxScaler())
])

In [6]:
dados_transformados = pd.DataFrame(num_pipe.fit_transform(dados))
dados_transformados.head()

,0,1,2,3,4,5,6
0,0.054054,0.00000,0.347222,0.066667,0.111111,0.139575,0.147390
1,0.148649,0.00464,0.930556,0.266667,0.148148,0.251931,0.090610
2,0.243243,0.00000,0.861111,0.266667,0.185185,0.295560,0.111274
3,0.337838,0.00000,0.291667,0.133333,0.111111,0.146139,0.091465
4,0.135135,0.16378,0.861111,0.200000,0.148148,0.239189,0.114702


#### ColumnTransformer

In [ ]:
cols_list_quali= ['FLAG_ELETRONICOS_3M',
                 'SATISFACAO_ULTIMA_COMPRA']

In [ ]:
FeatureEng = ColumnTransformer(
    transformers=[
        ('cat_ohe', OneHotEncoder(), cols_list_quali),
        ('num_pipe', num_pipe, cols_list_quanti)
    ]
)
pd.DataFrame(FeatureEng.fit_transform(dados))

#### Como aplicar

In [7]:
from sklearn.model_selection import RandomizedSearchCV                                                
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from scipy.stats import loguniform 
import warnings
warnings.filterwarnings('ignore')

In [8]:
lista_X_quanti = ['IDADE_CLIENTE',
                  'RENDA_MENSAL_CLIENTE',
                  'BEHAVIOUR_SCORE_CLIENTE',
                  'QTD_TRANSACOES_3M',
                  'QTD_ITENS_3M',
                  'VALOR_GASTO_3M',
                  'TICKET_MEDIO_3M']

lista_X_quali = ['FLAG_ELETRONICOS_3M',
                 'SATISFACAO_ULTIMA_COMPRA']

In [9]:
y = dados['VALOR_GASTO_PROX_12M']
X = dados[lista_X_quanti + lista_X_quali]

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=123)                                                        

In [10]:
#X_teste.to_csv("X_teste.csv", index=False)

In [11]:
class Wrapper(BaseEstimator):
    def __init__(self, intermediate_model):                
        self.intermediate_model = intermediate_model
    def fit(self, X, y=None):                        
        return self                                  
    def transform(self, X_teste):
        return self.intermediate_model.predict(X_teste)

In [12]:
preprocessador = ColumnTransformer(transformers=[
    ("quanti", StandardScaler(), lista_X_quanti),
    ("quali", OneHotEncoder(
        sparse_output=False,drop="first", handle_unknown='ignore'),lista_X_quali)
])

In [13]:
RL = Pipeline([
        ("preprocess", preprocessador),
        ("linear regression", RandomizedSearchCV(estimator=LinearRegression(),
                  param_distributions={},
                  scoring='neg_root_mean_squared_log_error',
                  cv=10) )
])

In [14]:
RD =  Pipeline([
        ("preprocess", preprocessador),
        ("linear regression", RandomizedSearchCV(estimator=Ridge(),
                  param_distributions={'alpha': loguniform(1e-5, 1e1) },
                  scoring='neg_root_mean_squared_log_error',
                  cv=10) )
])

In [15]:
treino_predict_pipe = FeatureUnion([
    ('linear', Wrapper(RL.fit(X_treino, y_treino))),
    ('ridge', Wrapper(RD.fit(X_treino, y_treino)))
])

In [16]:
y_hat = treino_predict_pipe.fit_transform(X_teste)
y_hat[:10]

array([ 584.75055789, 5481.49951283, 4338.86026119, 2023.84551492,
       3137.28878541, 5922.15906779, 1843.64941438, 4680.58782678,
       1788.20394403, 1585.34864919])

In [17]:
model_select = int(len(y_hat)/2)

In [18]:
from sklearn.metrics import root_mean_squared_error
print(root_mean_squared_error(y_teste, y_hat[:model_select]))
print(root_mean_squared_error(y_teste, y_hat[model_select:]))

696.79226412171
696.4215975105282
